In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

import scanpy as sc
sc.settings.n_jobs = 56
sc.settings.set_figure_params(dpi=180, dpi_save=300, frameon=False, figsize=(4, 4), fontsize=8, facecolor='white')

In [ ]:
def calc_spatial_fingerprint_df_2d(adata_query, adata_ref, cell_type_col, 
                              slice_col='slice_id', xy_cols=['global_x', 'global_y'],
                              K = 50, verbose=False):
    
    # Convert the cell types to indices in the neighborhood vector
    ct_to_neighbor_id_dict = {}

    unique_cts = np.unique(adata_ref.obs[cell_type_col])
    
    for i in range(len(unique_cts)):
        ct_to_neighbor_id_dict[unique_cts[i]] = i

    L_neighbor_vec = len(unique_cts)
    
    # Generate a matrix of neighborhood cell types
    all_knn_neighor_vecs = np.zeros((adata_query.shape[0], L_neighbor_vec), dtype=np.float32)


    # Iterate through the slices
    slice_ids = np.unique(adata_query.obs[slice_col])
    for slice_id in slice_ids:
        if verbose:
            print(f'Generate neighbor vectors for slice {slice_id}')
    
        slice_mask_q = adata_query.obs[slice_col] == slice_id
        df_slice_q = adata_query.obs[slice_mask_q]
        slice_mask_r = adata_ref.obs[slice_col] == slice_id
        df_slice_r = adata_ref.obs[slice_mask_r]
    
        # Skip the slices if there is not enough reference cells
        if df_slice_r.shape[0] < K:
            continue
    
        # Calculate spatial K nearest neighbors
        X_spatial_query = np.array(df_slice_q[xy_cols])
        X_spatial_ref = np.array(df_slice_r[xy_cols])

        nbrs = NearestNeighbors(n_neighbors=K, algorithm='ball_tree').fit(X_spatial_ref)
        distances, indices = nbrs.kneighbors(X_spatial_query)
        if verbose:
            print('Mean KNN distance =', np.mean(distances))
    
        # Get the cell types of each cell's KNN
        knn_cts = np.array(df_slice_r[cell_type_col])[indices]
    
        # Generate the matrix of neighbor cell types
        knn_neighbor_vecs = np.zeros((df_slice_q.shape[0], L_neighbor_vec))
        for i in range(df_slice_q.shape[0]):
            
            # Estimate the charasteristic neighbor distance
            neighbor_R0 = np.quantile(distances[i], max(0.1, 5 / K))
            for j in range(K):
                ct_neighbor_id = ct_to_neighbor_id_dict[knn_cts[i, j]]
            
                knn_neighbor_vecs[i, ct_neighbor_id] += np.exp(- (distances[i, j] / neighbor_R0) ** 2)
            
        all_knn_neighor_vecs[slice_mask_q] = knn_neighbor_vecs
        
    return pd.DataFrame(all_knn_neighor_vecs, adata_query.obs.index, list(unique_cts))

def merge_clusters(obs_df, cluster_col, clusters_to_merge):
    merged_cluster_id = sorted(clusters_to_merge)[0]
    obs_df[cluster_col][obs_df[cluster_col].isin(clusters_to_merge)] = merged_cluster_id
    obs_df[cluster_col] = obs_df[cluster_col].cat.remove_unused_categories()

In [ ]:
# Load the data
adata = sc.read_h5ad('adata_spatial_modules_level_1_annotated.h5ad')
adata

In [ ]:
# Only keep the neurons
all_subclasses = np.unique(adata.obs['subclass_label_transfer'])
neuron_subclasses = [s for s in all_subclasses if not s.endswith('NN')]

adata = adata[adata.obs['subclass_label_transfer'].isin(neuron_subclasses)]
adata

In [ ]:
sm1_names = ['SM_MOB outer', 'SM_MOB inner', 'SM_CTX', 'SM_STR', 'SM_PAL/STR', 'SM_OLF/HPF',
    'SM_HY', 'SM_TH', 'SM_MB', 'SM_HB', 'SM_CB', ]
output_path = 'spatial_module_2_adatas'
os.makedirs(output_path, exist_ok=True)

In [ ]:
%%time
# Parameters
sm1n = 'SM_MOB outer'
n_pcs = 50
n_neighbors = 30
resolution = 0.2
example_slice_ids = ['co2_slice5', 'co2_slice15', 'co2_slice25', 'sa1_slice6']

# Generate spatial fingerprints
adata_sm1 = adata[adata.obs['spatial_modules_level_1_name'] == sm1n]

subclass_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'subclass_label_transfer',
                                                                xy_cols=['center_x', 'center_y'])
cluster_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'cl_transfer',
                                                                xy_cols=['center_x', 'center_y'])  

spatial_fingerprint_df = subclass_spatial_fingerprint_df.merge(cluster_spatial_fingerprint_df, 
                                      left_index=True, right_index=True)


adata_sm1_subclass_sf = sc.AnnData(spatial_fingerprint_df)
adata_sm1_subclass_sf.obs = adata_sm1.obs.copy()
adata_sm1_subclass_sf = adata_sm1_subclass_sf[spatial_fingerprint_df.values.sum(axis=1) > 0]

# Normalize the fingerprint vectors
fp_norms = np.linalg.norm(adata_sm1_subclass_sf.X, axis=1)
adata_sm1_subclass_sf.X = adata_sm1_subclass_sf.X / fp_norms[:, np.newaxis]

# Clustering
n_pcs = min(n_pcs, adata_sm1_subclass_sf.shape[1] - 1)
sc.tl.pca(adata_sm1_subclass_sf, svd_solver='arpack', n_comps=n_pcs)
sc.pp.neighbors(adata_sm1_subclass_sf, n_neighbors=n_neighbors, n_pcs=n_pcs)
sc.tl.leiden(adata_sm1_subclass_sf, resolution=resolution)

# UMAP plot
sc.tl.paga(adata_sm1_subclass_sf, groups='leiden')
sc.pl.paga(adata_sm1_subclass_sf)
sc.tl.umap(adata_sm1_subclass_sf, min_dist=0.2, init_pos='paga')
sc.pl.umap(adata_sm1_subclass_sf, color='leiden', legend_loc='on data')
            
# Spatial plot
color_vec = np.array(['#000000'] * adata_sm1_subclass_sf.shape[0])
for i in range(len(adata_sm1_subclass_sf.obs['leiden'].cat.categories)):
    color_vec[adata_sm1_subclass_sf.obs['leiden'] == adata_sm1_subclass_sf.obs['leiden'].cat.categories[i]] =\
            adata_sm1_subclass_sf.uns['leiden_colors'][i]
    adata_sm1_subclass_sf.obs['color_leiden'] = color_vec

for example_slice_id in example_slice_ids:
    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
    adata_tmp = adata_sm1_subclass_sf[adata_sm1_subclass_sf.obs['slice_id'] == example_slice_id]
    ax.scatter(adata_tmp.obs['center_x'], adata_tmp.obs['center_y'], c=adata_tmp.obs['color_leiden'], 
               s=2, marker='.', edgecolor='none', rasterized=True)
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.set_title(example_slice_id)
    
adata_sm1_subclass_sf.write(os.path.join(output_path, sm1n.replace('/', '-') + '.h5ad'))

In [ ]:
%%time
# Parameters
sm1n = 'SM_MOB inner'
n_pcs = 50
n_neighbors = 30
resolution = 0.2
example_slice_ids = ['co2_slice5', 'co2_slice15', 'co2_slice25', 'sa1_slice6']

# Generate spatial fingerprints
adata_sm1 = adata[adata.obs['spatial_modules_level_1_name'] == sm1n]

subclass_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'subclass_label_transfer',
                                                                xy_cols=['center_x', 'center_y'])
cluster_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'cl_transfer',
                                                                xy_cols=['center_x', 'center_y'])  

spatial_fingerprint_df = subclass_spatial_fingerprint_df.merge(cluster_spatial_fingerprint_df, 
                                      left_index=True, right_index=True)


adata_sm1_subclass_sf = sc.AnnData(spatial_fingerprint_df)
adata_sm1_subclass_sf.obs = adata_sm1.obs.copy()
adata_sm1_subclass_sf = adata_sm1_subclass_sf[spatial_fingerprint_df.values.sum(axis=1) > 0]

# Normalize the fingerprint vectors
fp_norms = np.linalg.norm(adata_sm1_subclass_sf.X, axis=1)
adata_sm1_subclass_sf.X = adata_sm1_subclass_sf.X / fp_norms[:, np.newaxis]

# Clustering
n_pcs = min(n_pcs, adata_sm1_subclass_sf.shape[1] - 1)
sc.tl.pca(adata_sm1_subclass_sf, svd_solver='arpack', n_comps=n_pcs)
sc.pp.neighbors(adata_sm1_subclass_sf, n_neighbors=n_neighbors, n_pcs=n_pcs)
sc.tl.leiden(adata_sm1_subclass_sf, resolution=resolution)

# UMAP plot
sc.tl.paga(adata_sm1_subclass_sf, groups='leiden')
sc.pl.paga(adata_sm1_subclass_sf)
sc.tl.umap(adata_sm1_subclass_sf, min_dist=0.2, init_pos='paga')
sc.pl.umap(adata_sm1_subclass_sf, color='leiden', legend_loc='on data')
            
# Spatial plot
color_vec = np.array(['#000000'] * adata_sm1_subclass_sf.shape[0])
for i in range(len(adata_sm1_subclass_sf.obs['leiden'].cat.categories)):
    color_vec[adata_sm1_subclass_sf.obs['leiden'] == adata_sm1_subclass_sf.obs['leiden'].cat.categories[i]] =\
            adata_sm1_subclass_sf.uns['leiden_colors'][i]
    adata_sm1_subclass_sf.obs['color_leiden'] = color_vec

for example_slice_id in example_slice_ids:
    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
    adata_tmp = adata_sm1_subclass_sf[adata_sm1_subclass_sf.obs['slice_id'] == example_slice_id]
    ax.scatter(adata_tmp.obs['center_x'], adata_tmp.obs['center_y'], c=adata_tmp.obs['color_leiden'], 
               s=2, marker='.', edgecolor='none', rasterized=True)
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.set_title(example_slice_id)
    
adata_sm1_subclass_sf.write(os.path.join(output_path, sm1n.replace('/', '-') + '.h5ad'))

In [ ]:
%%time
# Parameters
sm1n = 'SM_PAL/STR'
n_pcs = 50
n_neighbors = 30
resolution = 0.2
example_slice_ids = ['co2_slice47', 'co2_slice61', 'co2_slice70', 'sa1_slice6']

# Generate spatial fingerprints
adata_sm1 = adata[adata.obs['spatial_modules_level_1_name'] == sm1n]

subclass_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'subclass_label_transfer',
                                                                xy_cols=['center_x', 'center_y'])
cluster_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'cl_transfer',
                                                                xy_cols=['center_x', 'center_y'])  

spatial_fingerprint_df = subclass_spatial_fingerprint_df.merge(cluster_spatial_fingerprint_df, 
                                      left_index=True, right_index=True)


adata_sm1_subclass_sf = sc.AnnData(spatial_fingerprint_df)
adata_sm1_subclass_sf.obs = adata_sm1.obs.copy()
adata_sm1_subclass_sf = adata_sm1_subclass_sf[spatial_fingerprint_df.values.sum(axis=1) > 0]

# Normalize the fingerprint vectors
fp_norms = np.linalg.norm(adata_sm1_subclass_sf.X, axis=1)
adata_sm1_subclass_sf.X = adata_sm1_subclass_sf.X / fp_norms[:, np.newaxis]

# Clustering
n_pcs = min(n_pcs, adata_sm1_subclass_sf.shape[1] - 1)
sc.tl.pca(adata_sm1_subclass_sf, svd_solver='arpack', n_comps=n_pcs)
sc.pp.neighbors(adata_sm1_subclass_sf, n_neighbors=n_neighbors, n_pcs=n_pcs)
sc.tl.leiden(adata_sm1_subclass_sf, resolution=resolution)

# UMAP plot
sc.tl.paga(adata_sm1_subclass_sf, groups='leiden')
sc.pl.paga(adata_sm1_subclass_sf)
sc.tl.umap(adata_sm1_subclass_sf, min_dist=0.2, init_pos='paga')
sc.pl.umap(adata_sm1_subclass_sf, color='leiden', legend_loc='on data')
            
# Spatial plot
color_vec = np.array(['#000000'] * adata_sm1_subclass_sf.shape[0])
for i in range(len(adata_sm1_subclass_sf.obs['leiden'].cat.categories)):
    color_vec[adata_sm1_subclass_sf.obs['leiden'] == adata_sm1_subclass_sf.obs['leiden'].cat.categories[i]] =\
            adata_sm1_subclass_sf.uns['leiden_colors'][i]
    adata_sm1_subclass_sf.obs['color_leiden'] = color_vec

for example_slice_id in example_slice_ids:
    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
    adata_tmp = adata_sm1_subclass_sf[adata_sm1_subclass_sf.obs['slice_id'] == example_slice_id]
    ax.scatter(adata_tmp.obs['center_x'], adata_tmp.obs['center_y'], c=adata_tmp.obs['color_leiden'], 
               s=2, marker='.', edgecolor='none', rasterized=True)
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.set_title(example_slice_id)
    
adata_sm1_subclass_sf.write(os.path.join(output_path, sm1n.replace('/', '-') + '.h5ad'))

In [ ]:
%%time
# Parameters
sm1n = 'SM_STR'
n_pcs = 50
n_neighbors = 30
resolution = 0.2
example_slice_ids = ['co2_slice47', 'co2_slice61', 'co2_slice76', 'sa1_slice6']

# Generate spatial fingerprints
adata_sm1 = adata[adata.obs['spatial_modules_level_1_name'] == sm1n]

subclass_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'subclass_label_transfer',
                                                                xy_cols=['center_x', 'center_y'])
cluster_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'cl_transfer',
                                                                xy_cols=['center_x', 'center_y'])  

spatial_fingerprint_df = subclass_spatial_fingerprint_df.merge(cluster_spatial_fingerprint_df, 
                                      left_index=True, right_index=True)


adata_sm1_subclass_sf = sc.AnnData(spatial_fingerprint_df)
adata_sm1_subclass_sf.obs = adata_sm1.obs.copy()
adata_sm1_subclass_sf = adata_sm1_subclass_sf[spatial_fingerprint_df.values.sum(axis=1) > 0]

# Normalize the fingerprint vectors
fp_norms = np.linalg.norm(adata_sm1_subclass_sf.X, axis=1)
adata_sm1_subclass_sf.X = adata_sm1_subclass_sf.X / fp_norms[:, np.newaxis]

# Clustering
n_pcs = min(n_pcs, adata_sm1_subclass_sf.shape[1] - 1)
sc.tl.pca(adata_sm1_subclass_sf, svd_solver='arpack', n_comps=n_pcs)
sc.pp.neighbors(adata_sm1_subclass_sf, n_neighbors=n_neighbors, n_pcs=n_pcs)
sc.tl.leiden(adata_sm1_subclass_sf, resolution=resolution)

# UMAP plot
sc.tl.paga(adata_sm1_subclass_sf, groups='leiden')
sc.pl.paga(adata_sm1_subclass_sf)
sc.tl.umap(adata_sm1_subclass_sf, min_dist=0.2, init_pos='paga')
sc.pl.umap(adata_sm1_subclass_sf, color='leiden', legend_loc='on data')
            
# Spatial plot
color_vec = np.array(['#000000'] * adata_sm1_subclass_sf.shape[0])
for i in range(len(adata_sm1_subclass_sf.obs['leiden'].cat.categories)):
    color_vec[adata_sm1_subclass_sf.obs['leiden'] == adata_sm1_subclass_sf.obs['leiden'].cat.categories[i]] =\
            adata_sm1_subclass_sf.uns['leiden_colors'][i]
    adata_sm1_subclass_sf.obs['color_leiden'] = color_vec

for example_slice_id in example_slice_ids:
    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
    adata_tmp = adata_sm1_subclass_sf[adata_sm1_subclass_sf.obs['slice_id'] == example_slice_id]
    ax.scatter(adata_tmp.obs['center_x'], adata_tmp.obs['center_y'], c=adata_tmp.obs['color_leiden'], 
               s=2, marker='.', edgecolor='none', rasterized=True)
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.set_title(example_slice_id)
    
adata_sm1_subclass_sf.write(os.path.join(output_path, sm1n.replace('/', '-') + '.h5ad'))

In [ ]:
%%time
# Parameters
sm1n = 'SM_CTX'
n_pcs = 50
n_neighbors = 40
resolution = 0.2
example_slice_ids = ['co2_slice47', 'co2_slice76', 'co2_slice106', 'sa1_slice6']

# Generate spatial fingerprints
adata_sm1 = adata[adata.obs['spatial_modules_level_1_name'] == sm1n]

subclass_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'subclass_label_transfer',
                                                                xy_cols=['center_x', 'center_y'])
cluster_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'cl_transfer',
                                                                xy_cols=['center_x', 'center_y'])  

spatial_fingerprint_df = subclass_spatial_fingerprint_df.merge(cluster_spatial_fingerprint_df, 
                                      left_index=True, right_index=True)


adata_sm1_subclass_sf = sc.AnnData(spatial_fingerprint_df)
adata_sm1_subclass_sf.obs = adata_sm1.obs.copy()
adata_sm1_subclass_sf = adata_sm1_subclass_sf[spatial_fingerprint_df.values.sum(axis=1) > 0]

# Normalize the fingerprint vectors
fp_norms = np.linalg.norm(adata_sm1_subclass_sf.X, axis=1)
adata_sm1_subclass_sf.X = adata_sm1_subclass_sf.X / fp_norms[:, np.newaxis]

# Clustering
n_pcs = min(n_pcs, adata_sm1_subclass_sf.shape[1] - 1)
sc.tl.pca(adata_sm1_subclass_sf, svd_solver='arpack', n_comps=n_pcs)
sc.pp.neighbors(adata_sm1_subclass_sf, n_neighbors=n_neighbors, n_pcs=n_pcs)
sc.tl.leiden(adata_sm1_subclass_sf, resolution=resolution)

# UMAP plot
sc.tl.paga(adata_sm1_subclass_sf, groups='leiden')
sc.pl.paga(adata_sm1_subclass_sf)
sc.tl.umap(adata_sm1_subclass_sf, min_dist=0.2, init_pos='paga')
sc.pl.umap(adata_sm1_subclass_sf, color='leiden', legend_loc='on data')
            
# Spatial plot
color_vec = np.array(['#000000'] * adata_sm1_subclass_sf.shape[0])
for i in range(len(adata_sm1_subclass_sf.obs['leiden'].cat.categories)):
    color_vec[adata_sm1_subclass_sf.obs['leiden'] == adata_sm1_subclass_sf.obs['leiden'].cat.categories[i]] =\
            adata_sm1_subclass_sf.uns['leiden_colors'][i]
    adata_sm1_subclass_sf.obs['color_leiden'] = color_vec

for example_slice_id in example_slice_ids:
    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
    adata_tmp = adata_sm1_subclass_sf[adata_sm1_subclass_sf.obs['slice_id'] == example_slice_id]
    ax.scatter(adata_tmp.obs['center_x'], adata_tmp.obs['center_y'], c=adata_tmp.obs['color_leiden'], 
               s=2, marker='.', edgecolor='none', rasterized=True)
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.set_title(example_slice_id)
    
adata_sm1_subclass_sf.write(os.path.join(output_path, sm1n.replace('/', '-') + '.h5ad'))

In [ ]:
%%time
# Parameters
sm1n = 'SM_OLF/HPF'
n_pcs = 50
n_neighbors = 30
resolution = 0.3
example_slice_ids = ['co2_slice40', 'co2_slice85', 'co2_slice100', 'sa1_slice6']

# Generate spatial fingerprints
adata_sm1 = adata[adata.obs['spatial_modules_level_1_name'] == sm1n]

subclass_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'subclass_label_transfer',
                                                                xy_cols=['center_x', 'center_y'])
cluster_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'cl_transfer',
                                                                xy_cols=['center_x', 'center_y'])  

spatial_fingerprint_df = subclass_spatial_fingerprint_df.merge(cluster_spatial_fingerprint_df, 
                                      left_index=True, right_index=True)


adata_sm1_subclass_sf = sc.AnnData(spatial_fingerprint_df)
adata_sm1_subclass_sf.obs = adata_sm1.obs.copy()
adata_sm1_subclass_sf = adata_sm1_subclass_sf[spatial_fingerprint_df.values.sum(axis=1) > 0]

# Normalize the fingerprint vectors
fp_norms = np.linalg.norm(adata_sm1_subclass_sf.X, axis=1)
adata_sm1_subclass_sf.X = adata_sm1_subclass_sf.X / fp_norms[:, np.newaxis]

# Clustering
n_pcs = min(n_pcs, adata_sm1_subclass_sf.shape[1] - 1)
sc.tl.pca(adata_sm1_subclass_sf, svd_solver='arpack', n_comps=n_pcs)
sc.pp.neighbors(adata_sm1_subclass_sf, n_neighbors=n_neighbors, n_pcs=n_pcs)
sc.tl.leiden(adata_sm1_subclass_sf, resolution=resolution)

# UMAP plot
sc.tl.paga(adata_sm1_subclass_sf, groups='leiden')
sc.pl.paga(adata_sm1_subclass_sf)
sc.tl.umap(adata_sm1_subclass_sf, min_dist=0.2, init_pos='paga')
sc.pl.umap(adata_sm1_subclass_sf, color='leiden', legend_loc='on data')
            
# Spatial plot
color_vec = np.array(['#000000'] * adata_sm1_subclass_sf.shape[0])
for i in range(len(adata_sm1_subclass_sf.obs['leiden'].cat.categories)):
    color_vec[adata_sm1_subclass_sf.obs['leiden'] == adata_sm1_subclass_sf.obs['leiden'].cat.categories[i]] =\
            adata_sm1_subclass_sf.uns['leiden_colors'][i]
    adata_sm1_subclass_sf.obs['color_leiden'] = color_vec

for example_slice_id in example_slice_ids:
    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
    adata_tmp = adata_sm1_subclass_sf[adata_sm1_subclass_sf.obs['slice_id'] == example_slice_id]
    ax.scatter(adata_tmp.obs['center_x'], adata_tmp.obs['center_y'], c=adata_tmp.obs['color_leiden'], 
               s=2, marker='.', edgecolor='none', rasterized=True)
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.set_title(example_slice_id)
    
adata_sm1_subclass_sf.write(os.path.join(output_path, sm1n.replace('/', '-') + '.h5ad'))

In [ ]:
%%time
# Parameters
sm1n = 'SM_HY'
n_pcs = 50
n_neighbors = 30
resolution = 0.2
example_slice_ids = ['co2_slice67', 'co2_slice76', 'co2_slice85', 'sa1_slice6']

# Generate spatial fingerprints
adata_sm1 = adata[adata.obs['spatial_modules_level_1_name'] == sm1n]

subclass_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'subclass_label_transfer',
                                                                xy_cols=['center_x', 'center_y'])
cluster_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'cl_transfer',
                                                                xy_cols=['center_x', 'center_y'])  

spatial_fingerprint_df = subclass_spatial_fingerprint_df.merge(cluster_spatial_fingerprint_df, 
                                      left_index=True, right_index=True)


adata_sm1_subclass_sf = sc.AnnData(spatial_fingerprint_df)
adata_sm1_subclass_sf.obs = adata_sm1.obs.copy()
adata_sm1_subclass_sf = adata_sm1_subclass_sf[spatial_fingerprint_df.values.sum(axis=1) > 0]

# Normalize the fingerprint vectors
fp_norms = np.linalg.norm(adata_sm1_subclass_sf.X, axis=1)
adata_sm1_subclass_sf.X = adata_sm1_subclass_sf.X / fp_norms[:, np.newaxis]

# Clustering
n_pcs = min(n_pcs, adata_sm1_subclass_sf.shape[1] - 1)
sc.tl.pca(adata_sm1_subclass_sf, svd_solver='arpack', n_comps=n_pcs)
sc.pp.neighbors(adata_sm1_subclass_sf, n_neighbors=n_neighbors, n_pcs=n_pcs)
sc.tl.leiden(adata_sm1_subclass_sf, resolution=resolution)

# UMAP plot
sc.tl.paga(adata_sm1_subclass_sf, groups='leiden')
sc.pl.paga(adata_sm1_subclass_sf)
sc.tl.umap(adata_sm1_subclass_sf, min_dist=0.2, init_pos='paga')
sc.pl.umap(adata_sm1_subclass_sf, color='leiden', legend_loc='on data')
            
# Spatial plot
color_vec = np.array(['#000000'] * adata_sm1_subclass_sf.shape[0])
for i in range(len(adata_sm1_subclass_sf.obs['leiden'].cat.categories)):
    color_vec[adata_sm1_subclass_sf.obs['leiden'] == adata_sm1_subclass_sf.obs['leiden'].cat.categories[i]] =\
            adata_sm1_subclass_sf.uns['leiden_colors'][i]
    adata_sm1_subclass_sf.obs['color_leiden'] = color_vec

for example_slice_id in example_slice_ids:
    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
    adata_tmp = adata_sm1_subclass_sf[adata_sm1_subclass_sf.obs['slice_id'] == example_slice_id]
    ax.scatter(adata_tmp.obs['center_x'], adata_tmp.obs['center_y'], c=adata_tmp.obs['color_leiden'], 
               s=2, marker='.', edgecolor='none', rasterized=True)
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.set_title(example_slice_id)
    
adata_sm1_subclass_sf.write(os.path.join(output_path, sm1n.replace('/', '-') + '.h5ad'))

In [ ]:
%%time
# Parameters
sm1n = 'SM_TH'
n_pcs = 50
n_neighbors = 30
resolution = 0.2
example_slice_ids = ['co2_slice76', 'co2_slice85', 'co2_slice90', 'sa1_slice6']

# Generate spatial fingerprints
adata_sm1 = adata[adata.obs['spatial_modules_level_1_name'] == sm1n]

subclass_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'subclass_label_transfer',
                                                                xy_cols=['center_x', 'center_y'])
cluster_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'cl_transfer',
                                                                xy_cols=['center_x', 'center_y'])  

spatial_fingerprint_df = subclass_spatial_fingerprint_df.merge(cluster_spatial_fingerprint_df, 
                                      left_index=True, right_index=True)


adata_sm1_subclass_sf = sc.AnnData(spatial_fingerprint_df)
adata_sm1_subclass_sf.obs = adata_sm1.obs.copy()
adata_sm1_subclass_sf = adata_sm1_subclass_sf[spatial_fingerprint_df.values.sum(axis=1) > 0]

# Normalize the fingerprint vectors
fp_norms = np.linalg.norm(adata_sm1_subclass_sf.X, axis=1)
adata_sm1_subclass_sf.X = adata_sm1_subclass_sf.X / fp_norms[:, np.newaxis]

# Clustering
n_pcs = min(n_pcs, adata_sm1_subclass_sf.shape[1] - 1)
sc.tl.pca(adata_sm1_subclass_sf, svd_solver='arpack', n_comps=n_pcs)
sc.pp.neighbors(adata_sm1_subclass_sf, n_neighbors=n_neighbors, n_pcs=n_pcs)
sc.tl.leiden(adata_sm1_subclass_sf, resolution=resolution)

# UMAP plot
sc.tl.paga(adata_sm1_subclass_sf, groups='leiden')
sc.pl.paga(adata_sm1_subclass_sf)
sc.tl.umap(adata_sm1_subclass_sf, min_dist=0.2, init_pos='paga')
sc.pl.umap(adata_sm1_subclass_sf, color='leiden', legend_loc='on data')
            
# Spatial plot
color_vec = np.array(['#000000'] * adata_sm1_subclass_sf.shape[0])
for i in range(len(adata_sm1_subclass_sf.obs['leiden'].cat.categories)):
    color_vec[adata_sm1_subclass_sf.obs['leiden'] == adata_sm1_subclass_sf.obs['leiden'].cat.categories[i]] =\
            adata_sm1_subclass_sf.uns['leiden_colors'][i]
    adata_sm1_subclass_sf.obs['color_leiden'] = color_vec

for example_slice_id in example_slice_ids:
    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
    adata_tmp = adata_sm1_subclass_sf[adata_sm1_subclass_sf.obs['slice_id'] == example_slice_id]
    ax.scatter(adata_tmp.obs['center_x'], adata_tmp.obs['center_y'], c=adata_tmp.obs['color_leiden'], 
               s=2, marker='.', edgecolor='none', rasterized=True)
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.set_title(example_slice_id)
    
adata_sm1_subclass_sf.write(os.path.join(output_path, sm1n.replace('/', '-') + '.h5ad'))

In [ ]:
%%time
# Parameters
sm1n = 'SM_MB'
n_pcs = 50
n_neighbors = 30
resolution = 0.3
example_slice_ids = ['co2_slice90', 'co2_slice105', 'co2_slice119', 'sa1_slice6']

# Generate spatial fingerprints
adata_sm1 = adata[adata.obs['spatial_modules_level_1_name'] == sm1n]

subclass_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'subclass_label_transfer',
                                                                xy_cols=['center_x', 'center_y'])
cluster_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'cl_transfer',
                                                                xy_cols=['center_x', 'center_y'])  

spatial_fingerprint_df = subclass_spatial_fingerprint_df.merge(cluster_spatial_fingerprint_df, 
                                      left_index=True, right_index=True)


adata_sm1_subclass_sf = sc.AnnData(spatial_fingerprint_df)
adata_sm1_subclass_sf.obs = adata_sm1.obs.copy()
adata_sm1_subclass_sf = adata_sm1_subclass_sf[spatial_fingerprint_df.values.sum(axis=1) > 0]

# Normalize the fingerprint vectors
fp_norms = np.linalg.norm(adata_sm1_subclass_sf.X, axis=1)
adata_sm1_subclass_sf.X = adata_sm1_subclass_sf.X / fp_norms[:, np.newaxis]

# Clustering
n_pcs = min(n_pcs, adata_sm1_subclass_sf.shape[1] - 1)
sc.tl.pca(adata_sm1_subclass_sf, svd_solver='arpack', n_comps=n_pcs)
sc.pp.neighbors(adata_sm1_subclass_sf, n_neighbors=n_neighbors, n_pcs=n_pcs)
sc.tl.leiden(adata_sm1_subclass_sf, resolution=resolution)

# UMAP plot
sc.tl.paga(adata_sm1_subclass_sf, groups='leiden')
sc.pl.paga(adata_sm1_subclass_sf)
sc.tl.umap(adata_sm1_subclass_sf, min_dist=0.2, init_pos='paga')
sc.pl.umap(adata_sm1_subclass_sf, color='leiden', legend_loc='on data')
            
# Spatial plot
color_vec = np.array(['#000000'] * adata_sm1_subclass_sf.shape[0])
for i in range(len(adata_sm1_subclass_sf.obs['leiden'].cat.categories)):
    color_vec[adata_sm1_subclass_sf.obs['leiden'] == adata_sm1_subclass_sf.obs['leiden'].cat.categories[i]] =\
            adata_sm1_subclass_sf.uns['leiden_colors'][i]
    adata_sm1_subclass_sf.obs['color_leiden'] = color_vec

for example_slice_id in example_slice_ids:
    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
    adata_tmp = adata_sm1_subclass_sf[adata_sm1_subclass_sf.obs['slice_id'] == example_slice_id]
    ax.scatter(adata_tmp.obs['center_x'], adata_tmp.obs['center_y'], c=adata_tmp.obs['color_leiden'], 
               s=2, marker='.', edgecolor='none', rasterized=True)
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.set_title(example_slice_id)
    
adata_sm1_subclass_sf.write(os.path.join(output_path, sm1n.replace('/', '-') + '.h5ad'))

In [ ]:
%%time
# Parameters
sm1n = 'SM_HB'
n_pcs = 50
n_neighbors = 30
resolution = 0.2
example_slice_ids = ['co2_slice100', 'co2_slice119', 'co2_slice141', 'sa1_slice6']

# Generate spatial fingerprints
adata_sm1 = adata[adata.obs['spatial_modules_level_1_name'] == sm1n]

subclass_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'subclass_label_transfer',
                                                                xy_cols=['center_x', 'center_y'])
cluster_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'cl_transfer',
                                                                xy_cols=['center_x', 'center_y'])  

spatial_fingerprint_df = subclass_spatial_fingerprint_df.merge(cluster_spatial_fingerprint_df, 
                                      left_index=True, right_index=True)


adata_sm1_subclass_sf = sc.AnnData(spatial_fingerprint_df)
adata_sm1_subclass_sf.obs = adata_sm1.obs.copy()
adata_sm1_subclass_sf = adata_sm1_subclass_sf[spatial_fingerprint_df.values.sum(axis=1) > 0]

# Normalize the fingerprint vectors
fp_norms = np.linalg.norm(adata_sm1_subclass_sf.X, axis=1)
adata_sm1_subclass_sf.X = adata_sm1_subclass_sf.X / fp_norms[:, np.newaxis]

# Clustering
n_pcs = min(n_pcs, adata_sm1_subclass_sf.shape[1] - 1)
sc.tl.pca(adata_sm1_subclass_sf, svd_solver='arpack', n_comps=n_pcs)
sc.pp.neighbors(adata_sm1_subclass_sf, n_neighbors=n_neighbors, n_pcs=n_pcs)
sc.tl.leiden(adata_sm1_subclass_sf, resolution=resolution)

# UMAP plot
sc.tl.paga(adata_sm1_subclass_sf, groups='leiden')
sc.pl.paga(adata_sm1_subclass_sf)
sc.tl.umap(adata_sm1_subclass_sf, min_dist=0.2, init_pos='paga')
sc.pl.umap(adata_sm1_subclass_sf, color='leiden', legend_loc='on data')
            
# Spatial plot
color_vec = np.array(['#000000'] * adata_sm1_subclass_sf.shape[0])
for i in range(len(adata_sm1_subclass_sf.obs['leiden'].cat.categories)):
    color_vec[adata_sm1_subclass_sf.obs['leiden'] == adata_sm1_subclass_sf.obs['leiden'].cat.categories[i]] =\
            adata_sm1_subclass_sf.uns['leiden_colors'][i]
    adata_sm1_subclass_sf.obs['color_leiden'] = color_vec

for example_slice_id in example_slice_ids:
    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
    adata_tmp = adata_sm1_subclass_sf[adata_sm1_subclass_sf.obs['slice_id'] == example_slice_id]
    ax.scatter(adata_tmp.obs['center_x'], adata_tmp.obs['center_y'], c=adata_tmp.obs['color_leiden'], 
               s=2, marker='.', edgecolor='none', rasterized=True)
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.set_title(example_slice_id)
    
adata_sm1_subclass_sf.write(os.path.join(output_path, sm1n.replace('/', '-') + '.h5ad'))

In [ ]:
%%time
# Parameters
sm1n = 'SM_CB'
n_pcs = 50
n_neighbors = 30
resolution = 0.2
example_slice_ids = ['co2_slice117', 'co2_slice126', 'co2_slice139', 'sa1_slice6']

# Generate spatial fingerprints
adata_sm1 = adata[adata.obs['spatial_modules_level_1_name'] == sm1n]

subclass_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'subclass_label_transfer',
                                                                xy_cols=['center_x', 'center_y'])
cluster_spatial_fingerprint_df = calc_spatial_fingerprint_df_2d(adata_sm1, adata_sm1, 'cl_transfer',
                                                                xy_cols=['center_x', 'center_y'])  

spatial_fingerprint_df = subclass_spatial_fingerprint_df.merge(cluster_spatial_fingerprint_df, 
                                      left_index=True, right_index=True)


adata_sm1_subclass_sf = sc.AnnData(spatial_fingerprint_df)
adata_sm1_subclass_sf.obs = adata_sm1.obs.copy()
adata_sm1_subclass_sf = adata_sm1_subclass_sf[spatial_fingerprint_df.values.sum(axis=1) > 0]

# Normalize the fingerprint vectors
fp_norms = np.linalg.norm(adata_sm1_subclass_sf.X, axis=1)
adata_sm1_subclass_sf.X = adata_sm1_subclass_sf.X / fp_norms[:, np.newaxis]

# Clustering
n_pcs = min(n_pcs, adata_sm1_subclass_sf.shape[1] - 1)
sc.tl.pca(adata_sm1_subclass_sf, svd_solver='arpack', n_comps=n_pcs)
sc.pp.neighbors(adata_sm1_subclass_sf, n_neighbors=n_neighbors, n_pcs=n_pcs)
sc.tl.leiden(adata_sm1_subclass_sf, resolution=resolution)

# UMAP plot
sc.tl.paga(adata_sm1_subclass_sf, groups='leiden')
sc.pl.paga(adata_sm1_subclass_sf)
sc.tl.umap(adata_sm1_subclass_sf, min_dist=0.2, init_pos='paga')
sc.pl.umap(adata_sm1_subclass_sf, color='leiden', legend_loc='on data')
            
# Spatial plot
color_vec = np.array(['#000000'] * adata_sm1_subclass_sf.shape[0])
for i in range(len(adata_sm1_subclass_sf.obs['leiden'].cat.categories)):
    color_vec[adata_sm1_subclass_sf.obs['leiden'] == adata_sm1_subclass_sf.obs['leiden'].cat.categories[i]] =\
            adata_sm1_subclass_sf.uns['leiden_colors'][i]
    adata_sm1_subclass_sf.obs['color_leiden'] = color_vec

for example_slice_id in example_slice_ids:
    fig, ax = plt.subplots(figsize=(3, 3), dpi=150)
    adata_tmp = adata_sm1_subclass_sf[adata_sm1_subclass_sf.obs['slice_id'] == example_slice_id]
    ax.scatter(adata_tmp.obs['center_x'], adata_tmp.obs['center_y'], c=adata_tmp.obs['color_leiden'], 
               s=2, marker='.', edgecolor='none', rasterized=True)
    ax.set_aspect('equal')
    ax.set_axis_off()
    ax.set_title(example_slice_id)
    
adata_sm1_subclass_sf.write(os.path.join(output_path, sm1n.replace('/', '-') + '.h5ad'))